1° passo: extrair as características da imagens da base de treinamento

Para cada imagem da base de dados, iremos extrair os Momentos de Hu (que são 7), e no final do vetor, colocaremos a classe que pertence a imagem

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

files_path = [os.path.abspath(x) for x in os.listdir('./') if x.endswith('.png')]

def extrair_caracteristica(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    _, imgBinaria = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY) #seleciona apenas os pixels dentro do intervalo [250,255]
    momentos = cv2.moments(imgBinaria)
    momentosDeHu = cv2.HuMoments(momentos)
    feature = (-np.sign(momentosDeHu) * np.log10(np.abs(momentosDeHu)))
    return feature


base_teste = []

#extraindo as características das imagens na base de dados
for i in files_path:
    diretorio, arquivo = os.path.split(i)
    imagem = cv2.imread(arquivo)
    carac = extrair_caracteristica(imagem)
    classe = arquivo.split('-')
    base_teste.append((carac, classe[0]))   

In [2]:
print(base_teste)

[(array([[  2.54496897],
       [  5.41237509],
       [  9.36688516],
       [  9.68917536],
       [ 19.9660983 ],
       [-13.03176335],
       [-19.22421959]]), '3'), (array([[  2.901805  ],
       [  6.13669137],
       [  9.26275766],
       [  9.98309001],
       [ 19.66622239],
       [ 13.05503555],
       [-19.91397104]]), '1'), (array([[  2.71583903],
       [  5.84826526],
       [  9.29749338],
       [  9.90277278],
       [-19.50476404],
       [-12.83892137],
       [-20.53766721]]), '3'), (array([[  2.78786451],
       [  5.92376711],
       [ 10.40973889],
       [ 10.75334106],
       [ 21.52360153],
       [ 15.24256795],
       [-21.45291745]]), '2'), (array([[  2.84454981],
       [  6.2452239 ],
       [ 11.0179522 ],
       [ 10.96156686],
       [ 22.00838906],
       [ 14.10686467],
       [-22.2694344 ]]), '2'), (array([[  2.79664963],
       [  6.04278819],
       [  9.72997911],
       [ 10.77209176],
       [ 21.1449219 ],
       [ 13.87478825],
       [-2

2° passo: calcular o vetor de característica da imagem consulta

In [3]:
img = cv2.imread('2-1.png')
vetor_consulta = extrair_caracteristica(img)

In [4]:
print(vetor_consulta)

[[  2.79664963]
 [  6.04278819]
 [  9.72997911]
 [ 10.77209176]
 [ 21.1449219 ]
 [ 13.87478825]
 [-21.20674854]]


3° passo: definir uma função de distância. Abaixo temos a distância Euclidiana

In [5]:
def distancia(a, b):
    M = len(a)
    soma = 0
    for i in range(M):
        soma = soma + ((a[i]-b[i])**2)
    return np.sqrt(soma) 

4° passo: calcular a distancia do vetor_consulta com todos os vetores das imagens da base_teste

OBSERVAÇÃO: após calcular distância, incluir a classe da imagem que foi calculada a distância

In [6]:
#calculando a distancia do vetor de características da imagem consulta com todos
# os vetores de características extraidos das imagens que estão na base de dados
d = []

for feat in base_teste:
    vetor = feat[0]
    dist = distancia(vetor, vetor_consulta)
    d.append((dist, feat[1]))

In [7]:
print(d)

[(array([27.037897]), '3'), (array([2.32174838]), '1'), (array([48.65649029]), '3'), (array([1.59733742]), '2'), (array([1.91488547]), '2'), (array([0.]), '2'), (array([26.97120339]), '3'), (array([5.25037655]), '1'), (array([2.5950679]), '1')]


5° passo: ordenar as distâncias em ordem crescente (menor para o maior)

In [8]:
e = sorted(d)

In [9]:
print(e)

[(array([0.]), '2'), (array([1.59733742]), '2'), (array([1.91488547]), '2'), (array([2.32174838]), '1'), (array([2.5950679]), '1'), (array([5.25037655]), '1'), (array([26.97120339]), '3'), (array([27.037897]), '3'), (array([48.65649029]), '3')]


6° passo: iremos contar qual é a classe que mais se repete considerando o top-k

Neste exemplo, iremos usar k = 3, ou seja, as 3 imagens mais similares

A classe que mais se repetir, será a classe da imagem consulta

In [10]:
k1 = e[0][1]
k2 = e[1][1]
k3 = e[2][1]
print(k1, k2, k3)

2 2 2


In [11]:
from statistics import mode
a = mode([k1,k2,k3])
print("classe final: ", a)

classe final:  2


## Código final, juntando todos os passos

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from statistics import mode

def distancia(a, b):
    M = len(a)
    soma = 0
    for i in range(M):
        soma = soma + ((a[i]-b[i])**2)
    return np.sqrt(soma) 

def extrair_caracteristica(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    _, imgBinaria = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY) #seleciona apenas os pixels dentro do intervalo [250,255]
    momentos = cv2.moments(imgBinaria)
    momentosDeHu = cv2.HuMoments(momentos)
    feature = (-np.sign(momentosDeHu) * np.log10(np.abs(momentosDeHu)))
    (contornos, tree) = cv2.findContours(imgBinaria, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contornos[0])
    
    Eixo1 = x +w
    Eixo2 = y+h
    
    if(Eixo1 < Eixo2):
        RA = Eixo1/Eixo2
        
    else:
        RA = Eixo2/Eixo1
    
    feature = np.append(feature,RA*10)
    return feature




files_path = [os.path.abspath(x) for x in os.listdir('./') if x.endswith('.png')]

base_teste = []

#extraindo as características das imagens na base de dados
for i in files_path:
    diretorio, arquivo = os.path.split(i)
    imagem = cv2.imread(arquivo)
    carac = extrair_caracteristica(imagem)
    classe = arquivo.split('-')
    base_teste.append((carac, classe[0]))   
    
    
img = cv2.imread('img_consulta.jpg')
vetor_consulta = extrair_caracteristica(img)    
    
#calculando a distancia do vetor de características da imagem consulta com todos
# os vetores de características extraidos das imagens que estão na base de dados
d = []

for feat in base_teste:
    vetor = feat[0]
    dist = distancia(vetor, vetor_consulta)
    d.append((dist, feat[1])) 
    
e = sorted(d)

k1 = e[0][1]
k2 = e[1][1]
k3 = e[2][1]

a = mode([k1,k2,k3])
print(k1, k2, k3)
print("classe final: ", a)


2 2 1
classe final:  2
